In [1]:
import pandas as pd
import numpy as np
import os
import django
from django.db.models import Max, Min, Avg, Q, F
from asgiref.sync import sync_to_async
import tqdm
from collections import defaultdict
from pandarallel import pandarallel
import requests
import json
from matplotlib import pyplot as plt

import ctypes
from ctypes import c_char_p, cdll
GoInt64 = ctypes.c_int64
GoInt = GoInt64
archive_node = "http://localhost:19545"

from etherscan.utils.parsing import ResponseParser as parser
pandarallel.initialize(progress_bar=True)
# os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
# os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
# django.setup()

from debtmonitor.models import *
from datavisualization.models import *
from datastorage.models import *
from debtmonitor.help_function import *

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


# Function

In [2]:
token_dict = dict(
    usdc = '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48',
    usdt = '0xdac17f958d2ee523a2206206994597c13d831ec7',
    dai =  '0x6b175474e89094c44da98b954eedeac495271d0f',
    # common collateral asset
    weth = "0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2"
)
decimal_dict = dict(
    usdc = 6,
    usdt = 6,
    dai =  18,
    # common collateral asset
    weth = 18
)

liquidation_threshold_dict = dict(
    usdc = 0.88,
    usdt = None, # not allowed as collateral
    dai =  0.8,
    # common collateral asset
    weth = 0.85
)
revert_token_dict = {v: k for k, v in token_dict.items()}

@sync_to_async
def get_potential_target():
    data = pd.DataFrame(
        list(
            LendingPoolInteraction.objects.all().values()
        )
    )
    return data['on_behalf_of'].unique().tolist()

@sync_to_async
def get_interaction_data(target):
    data = pd.DataFrame(
        list(
            LendingPoolInteraction.objects.filter(
                on_behalf_of=target
            ).annotate(
                 block_num=F('block_number__number'),
            ).all().values()
        )
    )
    return data

@sync_to_async
def get_reserves_status():
    data = pd.DataFrame(
        list(
            ReservesStatus.objects.annotate(
                 block_num=F('block_number__number'),
            ).all().values()
        )
    )
    return data

@sync_to_async
def get_liquidation_call(target=None):
    if target is None:
        data = pd.DataFrame(
            list(
                LiquidationCall.objects.annotate(
                    block_num=F('block_number__number'),
                ).all().values()
            )
        )
    else:
        data = pd.DataFrame(
            list(
                LiquidationCall.objects.filter(
                    on_behalf_of=target
                ).annotate(
                    block_num=F('block_number__number'),
                ).all().values()
            )
        )
    return data

@sync_to_async
def get_price_data(until_block_num, previous_block = 6424):
    ttt = pd.DataFrame(
        list(
            BlockPrice.objects.filter(
                Q(block_number__number__lte=until_block_num) &
                Q(block_number__number__gte=(until_block_num - previous_block))
            ).annotate(
                block_num=F('block_number__number'),
                oracle_name=F('token_pair__oracle__name'),
                token0 = F('token_pair__token0'),
                token1 = F('token_pair__token1'),
            ).all().values()
        )
    )
    return ttt.sort_values("block_num")

def invert_transformation(df_train, df_forecast):
    """Revert back the differencing to get the forecast to original scale."""
    df_fc = df_forecast.copy()
    columns = df_train.columns
    for col in columns:        
        # Roll back 1st Diff
        df_fc[str(col)] = (df_train[col].iloc[-1] + df_fc[col].cumsum()) # np.exp(df_train[col].iloc[-1] + df_fc[col].cumsum())
    return df_fc

def cal_hf(price_prediction, token_value_dicts, liquidation_threshold_dict):
    hf_list = []
    for i in range(price_prediction.shape[0]):
        collatearl_m_threshold_in_eth = 0
        debt_m_threshold_in_eth = 0
        for token_name, token_amount in token_value_dicts['collateral'].items():
            if token_name == 'weth':
                collatearl_m_threshold_in_eth += token_amount * liquidation_threshold_dict[token_name]
            else:
                collatearl_m_threshold_in_eth += token_amount * price_prediction['chainlink_' + token_name].loc[i]  * liquidation_threshold_dict[token_name]

        for token_name, token_amount in token_value_dicts['var_debt'].items():
            if token_name == 'weth':
                debt_m_threshold_in_eth += token_amount
            else:
                debt_m_threshold_in_eth += token_amount * price_prediction['chainlink_' + token_name].loc[i]

        for token_name, token_amount in token_value_dicts['sta_debt'].items():
            if token_name == 'weth':
                debt_m_threshold_in_eth += token_amount
            else:
                debt_m_threshold_in_eth += token_amount * price_prediction['chainlink_' + token_name].loc[i]
        collatearl_m_threshold_in_eth, debt_m_threshold_in_eth

        hf = (collatearl_m_threshold_in_eth/debt_m_threshold_in_eth)
        hf_list.append(hf)
    return pd.Series(hf_list)#, columns=['hf'])

def cal_pct_be_liquidated(mc_simulation_row):
    return (mc_simulation_row < 1).sum()/mc_simulation_row.count()

combine_block_n_index = lambda x: int(str(x['block_num']) + str(x['index']).zfill(6))

change_token_address_to_name = lambda x: revert_token_dict[x] if x in revert_token_dict else x

In [3]:
async def get_evaluation_data(target_address):
    interaction_df = await get_interaction_data(target_address)
    return interaction_df[interaction_df['action'] == 'LiquidationCall'][['block_num', 'index', 'on_behalf_of']]

In [4]:
# await get_evaluation_data('0x0e4a72e59127e878b51c4784f56eb299077fd807')

# Main Code

In [5]:
potential_targets = pd.read_csv("../data/potential_targets.csv", index_col=0)
potential_targets.head()

,on_behalf_of
0,0x0e4a72e59127e878b51c4784f56eb299077fd807
1,0x7ea46eca49d9f65072e713a502408595e61d0646
2,0x2b6cbf131976b85a9a773738e7dd04c4f15fa2cd
3,0xa030a30acf9e2885ff128adbf8f7fab2db180462
4,0xc124090f7a368a1a6d0b96bd1c6a865d34edf6e1


In [6]:
read_from_csv = True
if read_from_csv:
    liquidation_evl_df = pd.read_csv('../data/evaluation_target.csv', index_col=0)
else:
    liquidation_list = []
    for i in tqdm.tqdm(potential_targets['on_behalf_of']):
        liquidation_list.append(await get_evaluation_data(i))
    liquidation_evl_df = pd.concat(liquidation_list)
    liquidation_evl_df = liquidation_evl_df.reset_index(drop=True)
    liquidation_evl_df.to_csv('../data/evaluation_target.csv')
liquidation_evl_df

,block_num,index,on_behalf_of
0,14723214,96,0x0e4a72e59127e878b51c4784f56eb299077fd807
1,14741923,490,0x0e4a72e59127e878b51c4784f56eb299077fd807
2,14745493,135,0x0e4a72e59127e878b51c4784f56eb299077fd807
3,14758898,169,0x0e4a72e59127e878b51c4784f56eb299077fd807
4,14847627,271,0x0e4a72e59127e878b51c4784f56eb299077fd807
...,...,...,...
5684,14852421,80,0xd1a81de1ba42f06ab9bc7e74dcbd915c03ff0b21
5685,14944393,43,0xd1a81de1ba42f06ab9bc7e74dcbd915c03ff0b21
5686,14977439,16,0xd1a81de1ba42f06ab9bc7e74dcbd915c03ff0b21
5687,15065073,32,0xd1a81de1ba42f06ab9bc7e74dcbd915c03ff0b21


In [9]:
async def collect_data(liquidation_evl_df, block_before=1000, mc_amount = 300, step = 100):
    return_data = []
    reserves_status = await get_reserves_status()
    reserves_status = reserves_status[[
            'reserve', 'block_num', 'index',  
            'liquidity_rate', 'stable_borrow_rate', 'variable_borrow_rate', 
            'liquidity_index','variable_borrow_index'
    ]].copy()
    reserves_status['block_n_index'] = reserves_status.apply(combine_block_n_index, axis=1)
    reserves_status = reserves_status.sort_values('block_n_index').reset_index(drop=True)
    reserves_status['reserve'] = reserves_status['reserve'].apply(change_token_address_to_name).reset_index(drop=True)
    for iii in tqdm.tqdm(liquidation_evl_df.index):
        until_block_num = liquidation_evl_df.loc[iii, 'block_num'] - block_before
        until_index = 9999
        target_address = liquidation_evl_df.loc[iii, 'on_behalf_of']
        until_block_n_index = combine_block_n_index({'block_num': until_block_num, 'index': until_index})
        liquidation_index = until_index

        liquidation_df = await get_liquidation_call(target_address)
        interaction_df = await get_interaction_data(target_address)

        interaction_df = interaction_df['action block_num index on_behalf_of reserve amount rate_mode rate'.split(' ')].copy()
        
        liquidation_df = liquidation_df[[
            'block_num', 'index', 'on_behalf_of', 
            'collateral_asset', 'debt_asset', 'debt_to_cover', 'liquidated_collateral_amount',
            'liquidator', 'receive_atoken']].copy()

        interaction_df['block_n_index'] = interaction_df.apply(combine_block_n_index, axis=1)
        
        liquidation_df['block_n_index'] = liquidation_df.apply(combine_block_n_index, axis=1)

        interaction_df = interaction_df.sort_values('block_n_index').reset_index(drop=True)
        

        # just give a random reserve address, will be swich in the following part
        for index in interaction_df.index:
            if interaction_df.loc[index, 'action'] == "LiquidationCall":
                interaction_df.loc[index, 'reserve'] = '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'

        # merge
        interaction_df['reserve'] = interaction_df['reserve'].apply(change_token_address_to_name).reset_index(drop=True)
        
        user_df = interaction_df.merge(reserves_status, on=['reserve'], how='left')
        
        user_df['reserve'] = user_df['reserve'].apply(change_token_address_to_name).reset_index(drop=True)
        

        liquidation_df['collateral_asset'] = liquidation_df['collateral_asset'].apply(change_token_address_to_name)
        liquidation_df['debt_asset'] = liquidation_df['debt_asset'].apply(change_token_address_to_name)
        
        if 'usdt' in interaction_df[interaction_df['action']=='Deposit']['reserve'].to_list():
            continue

        def get_liquidation_data(df_row):
            df_row = df_row.copy()
            if df_row['action'] != 'LiquidationCall': return df_row
            # collateral
            block_n_index_x = df_row['block_n_index_x']
            liquidation_row = liquidation_df[liquidation_df['block_n_index'] == block_n_index_x]
            collateral_asset = liquidation_row['collateral_asset'].values[0]
            tmp_reserves_status = reserves_status[\
                (reserves_status['reserve'] == collateral_asset) &\
                (reserves_status['block_n_index'] <= block_n_index_x)].copy().sort_values('block_n_index')
            tmp_reserves_status = tmp_reserves_status.iloc[-1, :]

            df_row['block_num_y'] = tmp_reserves_status['block_num']
            df_row['index_y'] = tmp_reserves_status['index']
            df_row['liquidity_rate'] = tmp_reserves_status['liquidity_rate']
            df_row['liquidity_index'] = tmp_reserves_status['liquidity_index']
            df_row['block_n_index_y'] = tmp_reserves_status['block_n_index']

            debt_asset = liquidation_row['debt_asset'].values[0]
            tmp_reserves_status = reserves_status[\
                (reserves_status['reserve'] == debt_asset) &\
                (reserves_status['block_n_index'] <= block_n_index_x)].copy().sort_values('block_n_index')
            tmp_reserves_status = tmp_reserves_status.iloc[-1, :]

            df_row['stable_borrow_rate'] = tmp_reserves_status['stable_borrow_rate']
            df_row['variable_borrow_rate'] = tmp_reserves_status['variable_borrow_rate']
            df_row['variable_borrow_index'] = tmp_reserves_status['variable_borrow_index']
            
            return df_row

        from_df = user_df[user_df['block_n_index_y'] <= user_df['block_n_index_x']]
        from_df = from_df.loc[from_df.groupby('block_n_index_x').block_n_index_y.idxmax()].reset_index(drop=True)
        from_df = from_df.apply(get_liquidation_data, axis=1)

        collateral_dict = defaultdict(float)
        collatearl_able_dict = defaultdict(lambda :True)
        variable_debt_dict = defaultdict(float)
        stable_debt_dict = defaultdict(lambda : [None, None, None]) # amount, interest, start time

        liquidation_pool_target = [
            "ReserveUsedAsCollateralEnabled",
            "ReserveUsedAsCollateralDisabled", 
            "Deposit", 
            "Withdraw",
            "Borrow",
            "Repay",
            # "LiquidationCall",
            "Swap",
        ]

        SECONDS_PER_YEAR = 365 * 24 * 60 * 60
        RAY = 1e27

        ray_mul = lambda a,b: (a * b + RAY/2) / RAY
        ray_div = lambda a,b: (a * RAY + b/2) / b


        sub_interaction_df = interaction_df[interaction_df['block_n_index'] <= until_block_n_index].copy()

        __library = cdll.LoadLibrary('../eth_crawler/library.so')

        get_single_block_time = __library.get_single_block_time
        get_single_block_time.argtypes = [c_char_p, GoInt]
        get_single_block_time.restype = c_char_p

        # Block Time
        def get_block_time(block_num):
            try:
                res = get_single_block_time(
                    archive_node.encode(), 
                    GoInt(int(block_num))
                )
                res = res.decode("utf-8")
                res = json.loads(s=res)#.items()#, columns=['BlockNum', 'Timestamp'])
                
                return res[str(block_num)]
            except Exception as e: 
                print(e)


        def cal_stable_debt_change(stable_debt_amount_p, stable_borrow_rate_p, block_num, block_num_p):
            block_time = get_block_time(block_num)
            block_time_p = get_block_time(block_num_p)
            exp = block_time - block_time_p
            
            ###### Reference #####: https://etherscan.io/address/0xc6845a5c768bf8d7681249f8927877efda425baf#code
            expMinusOne = exp - 1
            expMinusTwo = exp - 2 if exp > 2 else 0
            ratePerSecond = stable_borrow_rate_p / SECONDS_PER_YEAR
            basePowerTwo = ray_mul(ratePerSecond, ratePerSecond) # (ratePerSecond * ratePerSecond + 0.5 * RAY)/RAY
            basePowerThree = ray_mul(basePowerTwo, ratePerSecond)#  + 0.5 * RAY)/RAY
            secondTerm = (exp * expMinusOne * basePowerTwo) / 2
            thirdTerm = (exp * expMinusOne * expMinusTwo * basePowerThree) / 6
            compounded_interest = RAY + (ratePerSecond * exp) + (secondTerm) + (thirdTerm)
            new_stable_balance = ray_mul(stable_debt_amount_p, compounded_interest)
            balance_increase = new_stable_balance - stable_debt_amount_p
            ########################################################################

            return new_stable_balance, balance_increase

        def update_target_debt_data(action_i, block_num, amount_i, token_name_i, 
                rate_mode_i, liquidity_index, variable_borrow_index, stable_borrow_rate):
            
            
            a_token_amount_i = ray_div(amount_i, liquidity_index)
            
            variable_debt_amount_i = ray_div(amount_i, variable_borrow_index)

            # For Stable Debt
            stable_debt_amount_i = amount_i #/ stable_borrow_rate
            stable_debt_amount_p, stable_borrow_rate_p, block_num_p = stable_debt_dict[token_name_i]
            if stable_debt_amount_p != None:
                new_stable_balance, balance_increase = cal_stable_debt_change(stable_debt_amount_p, stable_borrow_rate_p, block_num, block_num_p)

            
            if action_i == "ReserveUsedAsCollateralEnabled":
                collatearl_able_dict[token_name_i] = True
            elif action_i == "ReserveUsedAsCollateralDisabled":
                collatearl_able_dict[token_name_i] = False
            elif action_i == "Deposit":
                if collatearl_able_dict[token_name_i] == False and collateral_dict[token_name_i] == 0:
                    collatearl_able_dict[token_name_i] = True
                collateral_dict[token_name_i] += a_token_amount_i
            elif action_i == 'Withdraw':
                if (collateral_dict[token_name_i] - a_token_amount_i) < 0:
                    return False, np.abs(collateral_dict[token_name_i] - a_token_amount_i)
                collateral_dict[token_name_i] -= a_token_amount_i
            elif action_i == "Borrow":
                if rate_mode_i == '1': # stable
                    if stable_debt_dict[token_name_i][0] is None:
                        stable_debt_dict[token_name_i] = [stable_debt_amount_i, stable_borrow_rate, block_num]
                    else:
                        stable_debt_dict[token_name_i] = [new_stable_balance + stable_debt_amount_i, stable_borrow_rate, block_num]
                elif rate_mode_i == '2': # variable
                    variable_debt_dict[token_name_i] += variable_debt_amount_i
                else:
                    assert False, "rate_mode_i error"
            elif action_i == "Repay":
                if rate_mode_i == '1':
                    if (new_stable_balance - stable_debt_amount_i) < 0:
                        return False, np.abs(new_stable_balance - stable_debt_amount_i)
                    stable_debt_dict[token_name_i] = [new_stable_balance - stable_debt_amount_i, stable_borrow_rate, block_num]
                elif rate_mode_i == '2': # variable
                    if (variable_debt_dict[token_name_i] - variable_debt_amount_i) < 0:
                        return False, np.abs(variable_debt_dict[token_name_i] - variable_debt_amount_i)
                    variable_debt_dict[token_name_i] -= variable_debt_amount_i
            elif action_i == 'RebalanceStableBorrowRate':
                stable_debt_dict[token_name_i] = [new_stable_balance, stable_borrow_rate, block_num]
            else:
                assert False, "Interaction Data error"

            return True, 0

        def get_token_value(block_num, index, fix_decimal=False):
            collateral_in_original_unit = defaultdict(float)
            var_debt_in_original_unit = defaultdict(float)
            sta_debt_in_original_unit = defaultdict(float)

            block_n_index = combine_block_n_index(dict(block_num=block_num, index=index))

            for token_name, able in collatearl_able_dict.items():
                decimal_fixer = 1
                if able:
                    tmp_status = reserves_status[(reserves_status['reserve'] == token_name) &\
                        (reserves_status['block_n_index'] <= block_n_index)].copy().sort_values('block_n_index').iloc[-1,:]
                    if fix_decimal:
                        decimal_fixer = 10 ** decimal_dict[token_name]
                    collateral_in_original_unit[token_name] = ray_mul(collateral_dict[token_name], tmp_status["liquidity_index"]) / decimal_fixer
            
            for token_name, able in variable_debt_dict.items():
                decimal_fixer = 1
                tmp_status = reserves_status[(reserves_status['reserve'] == token_name) &\
                        (reserves_status['block_n_index'] <= block_n_index)].copy().sort_values('block_n_index').iloc[-1,:]
                if fix_decimal:
                    decimal_fixer = 10 ** decimal_dict[token_name]
                var_debt_in_original_unit[token_name] = ray_mul(variable_debt_dict[token_name], tmp_status["variable_borrow_index"]) / decimal_fixer

            for token_name, stable_debt in stable_debt_dict.items():
                decimal_fixer = 1
                if stable_debt[0] is not None:
                    stable_debt_amount_p, stable_borrow_rate_p, block_num_p = stable_debt_dict[token_name]
                    new_stable_balance, balance_increase = cal_stable_debt_change(stable_debt_amount_p, stable_borrow_rate_p, block_num, block_num_p)
                    if fix_decimal:
                        decimal_fixer = 10 ** decimal_dict[token_name]
                    sta_debt_in_original_unit[token_name] = new_stable_balance / decimal_fixer
            
            
            
            return collateral_in_original_unit, var_debt_in_original_unit, sta_debt_in_original_unit


        for index_i in sub_interaction_df.index:

            action_i = sub_interaction_df.loc[index_i, 'action']
            block_n_index = sub_interaction_df.loc[index_i, 'block_n_index']
            block_num = sub_interaction_df.loc[index_i, 'block_num']
            index = sub_interaction_df.loc[index_i, 'index']

            # block_time = get_block_time(block_num)
            before_data = from_df[from_df['block_n_index_x'] == block_n_index]#['amount'].values[0]
            liquidity_index = before_data['liquidity_index'].values[0]
            variable_borrow_index = before_data['variable_borrow_index'].values[0]
            stable_borrow_rate = before_data['stable_borrow_rate'].values[0]
            
            if action_i == "LiquidationCall":
                'collateral_asset', 'debt_asset', 'debt_to_cover', 'liquidated_collateral_amount',
                liquidation_i = liquidation_df[liquidation_df['block_n_index'] == block_n_index].copy().reset_index(drop=True)
                collateral_asset = liquidation_i.loc[0, 'collateral_asset']
                debt_asset = liquidation_i.loc[0, 'debt_asset']
                debt_to_cover = liquidation_i.loc[0, 'debt_to_cover']
                liquidated_collateral_amount = liquidation_i.loc[0, 'liquidated_collateral_amount']

                # a_token_amount_i = ray_div(liquidated_collateral_amount, liquidity_index)
                # collateral_dict[collateral_asset] -= a_token_amount_i

                collateral_in_original_unit, var_debt_in_original_unit, sta_debt_in_original_unit = get_token_value(block_num, index)

                if var_debt_in_original_unit[debt_asset] < debt_to_cover:
                    var_debt_to_liquidate = var_debt_in_original_unit[debt_asset]
                    sta_debt_to_repay = debt_to_cover - var_debt_to_liquidate
                else:
                    var_debt_to_liquidate = debt_to_cover
                    sta_debt_to_repay = 0

                success, remaining_token = update_target_debt_data(
                    "Repay", block_num, var_debt_to_liquidate, debt_asset, 
                    "2", liquidity_index, variable_borrow_index, stable_borrow_rate)
                assert success

                if sta_debt_to_repay > 0:
                    success, remaining_token = update_target_debt_data(
                        "Repay", block_num, sta_debt_to_repay, debt_asset, 
                        "1", liquidity_index, variable_borrow_index, stable_borrow_rate)
                    assert success
                
                success, remaining_token = update_target_debt_data(
                        "Withdraw", block_num, liquidated_collateral_amount, collateral_asset, 
                        "-1", liquidity_index, variable_borrow_index, stable_borrow_rate)
                assert success

            else:
                amount_i = sub_interaction_df.loc[index_i, 'amount']
                token_name_i = sub_interaction_df.loc[index_i, 'reserve']
                rate_mode_i = sub_interaction_df.loc[index_i, 'rate_mode']

                update_target_debt_data(action_i, block_num, amount_i, token_name_i, 
                rate_mode_i, liquidity_index, variable_borrow_index, stable_borrow_rate)
        token_value_dicts = get_token_value(until_block_num, liquidation_index, fix_decimal=True)
        token_value_dicts = {i:j for i,j in zip(['collateral', 'var_debt', 'sta_debt'], token_value_dicts)}


        price_data = await get_price_data(until_block_num, previous_block=6424)
        price_data['token0'] = price_data['token0'].apply(lambda x: 'weth' if x == 'eth' else x)
        price_data['token1'] = price_data['token1'].apply(lambda x: 'weth' if x == 'eth' else x)
        price_data = price_data[['block_num', 'oracle_name', 'token0', 'token1', 'current']]

        block_num_df = pd.DataFrame(
            range(
                price_data.block_num.min(), 
                until_block_num + 1
            ),
            columns=['block_num']
        )
        block_num_df.set_index('block_num', inplace=True)

        uniswapv3_price_dict = {}
        for token in ['usdt', 'dai', 'usdc']:
            sub_price_df = price_data[(price_data['oracle_name'] == 'uniswapv3') & (price_data['token1'] == token)].copy()
            sub_price_df[f'{token}'] = 1/sub_price_df['current']
            sub_price_df.set_index('block_num', inplace=True)
            sub_price_df = sub_price_df.merge(block_num_df, how='right', left_index=True, right_index=True)
            sub_price_df.fillna(method='ffill', inplace=True)
            sub_price_df.fillna(method='bfill', inplace=True)
            uniswapv3_price_dict[token] = sub_price_df[token]
        chainlink_price_dict ={}
        for token in ['usdt', 'dai', 'usdc']:
            sub_price_df = price_data[(price_data['oracle_name'] == 'chainlink') & (price_data['token0'] == token)].copy()
            sub_price_df[f'{token}'] = sub_price_df['current']
            sub_price_df.set_index('block_num', inplace=True)
            sub_price_df = sub_price_df.merge(block_num_df, how='right', left_index=True, right_index=True)
            sub_price_df.fillna(method='ffill', inplace=True)
            sub_price_df.fillna(method='bfill', inplace=True)
            chainlink_price_dict[token] = sub_price_df[token]
    

        collatearl_in_eth = 0
        debt_in_eth = 0
        for token_name, token_amount in token_value_dicts['collateral'].items():
            if token_name == 'weth':
                collatearl_in_eth += token_amount
            else:
                collatearl_in_eth += token_amount * chainlink_price_dict[token_name].loc[until_block_num]

        for token_name, token_amount in token_value_dicts['var_debt'].items():
            if token_name == 'weth':
                debt_in_eth += token_amount
            else:
                debt_in_eth += token_amount * chainlink_price_dict[token_name].loc[until_block_num]

        for token_name, token_amount in token_value_dicts['sta_debt'].items():
            if token_name == 'weth':
                debt_in_eth += token_amount
            else:
                debt_in_eth += token_amount * chainlink_price_dict[token_name].loc[until_block_num]

        collatearl_m_threshold_in_eth = 0
        debt_m_threshold_in_eth = 0
        for token_name, token_amount in token_value_dicts['collateral'].items():
            if token_name == 'weth':
                collatearl_m_threshold_in_eth += token_amount * liquidation_threshold_dict[token_name]
            else:
                if liquidation_threshold_dict[token_name] is None:
                    print(123)
                collatearl_m_threshold_in_eth += token_amount * chainlink_price_dict[token_name].loc[until_block_num]  * liquidation_threshold_dict[token_name]

        for token_name, token_amount in token_value_dicts['var_debt'].items():
            if token_name == 'weth':
                debt_m_threshold_in_eth += token_amount
            else:
                debt_m_threshold_in_eth += token_amount * chainlink_price_dict[token_name].loc[until_block_num]

        for token_name, token_amount in token_value_dicts['sta_debt'].items():
            if token_name == 'weth':
                debt_m_threshold_in_eth += token_amount
            else:
                debt_m_threshold_in_eth += token_amount * chainlink_price_dict[token_name].loc[until_block_num]
        collatearl_m_threshold_in_eth, debt_m_threshold_in_eth

        HF = (collatearl_m_threshold_in_eth/debt_m_threshold_in_eth)  


        used_token_list = []
        price_data_list = []
        price_name_list = []
        for asset_from in ['collateral', 'var_debt', 'sta_debt']:
            for token_name in ['usdc', 'usdt', 'dai']:
            # for token_name, token_amount in token_value_dicts[asset_from].items():
                if token_name == 'weth': continue
                if token_name not in used_token_list:
                    used_token_list.append(token_name)
                    price_data_list.append(chainlink_price_dict[token_name])
                    price_name_list.append(f'chainlink_{token_name}')
                    price_data_list.append(uniswapv3_price_dict[token_name])
                    price_name_list.append(f'uniswapv3_{token_name}')
        var_train_df = pd.concat(price_data_list, axis=1)
        var_train_df.columns = price_name_list
        var_train_df = var_train_df.reset_index(drop=False)

        train_test_split = 1
        train_data = var_train_df[:(int(len(var_train_df)*train_test_split)+1)].set_index('block_num')
        test_data = var_train_df[(int(len(var_train_df)*train_test_split)+1):].set_index('block_num')
        tmp_df = train_data.copy()#.set_index('block_num')
        # log_df = np.log(tmp_df)
        log_f_diff = tmp_df.diff().dropna() 
        log_f_diff = log_f_diff.reset_index(drop=True)
        trained_var = get_var_result(log_f_diff)              


        def mc_simulate(df, step=240):
            price_diff_prediction = pd.DataFrame(trained_var.simulate_var(step), columns=log_f_diff.columns)
            price_prediction = invert_transformation(tmp_df, price_diff_prediction) 
            hf_df = cal_hf(price_prediction, token_value_dicts, liquidation_threshold_dict)
            return hf_df
        
        
        mc_hf = pd.DataFrame(range(mc_amount)).apply(mc_simulate, args=(step,), axis=1).T
        potential_liquidation = mc_hf.apply(lambda x: (x < 1).any(), axis=0)
        mc_hf_pct = mc_hf.apply(cal_pct_be_liquidated, axis=1)

        # mc_hf = pd.DataFrame(range(mc_amount)).parallel_apply(mc_simulate, args=(step,), axis=1).T
        # potential_liquidation = mc_hf.parallel_apply(lambda x: (x < 1).any(), axis=0)
        # mc_hf_pct = mc_hf.parallel_apply(cal_pct_be_liquidated, axis=1)
        # print(tmp_df, token_value_dicts, liquidation_threshold_dict)
        actual_hf = cal_hf(tmp_df.reset_index(), token_value_dicts, liquidation_threshold_dict)

        return_data.append([liquidation_evl_df.loc[iii, :], mc_hf_pct, actual_hf])
        # break
    return return_data
ttt = await collect_data(liquidation_evl_df)

  0%|                         | 12/5689 [02:17<27:57:37, 17.73s/it]

In [ ]:
import pickle
with open('data/liquidation_evl.pickle', 'wb') as handle:
    pickle.dump(ttt, handle, protocol=pickle.HIGHEST_PROTOCOL)
ttt

[[block_num                                         14759384
  index                                                  475
  on_behalf_of    0x7ea46eca49d9f65072e713a502408595e61d0646
  Name: 5, dtype: object,
  0     0.0
  1     0.0
  2     0.0
  3     0.0
  4     0.0
       ... 
  95    0.0
  96    0.0
  97    0.0
  98    0.0
  99    0.0
  Length: 100, dtype: float64,
  0       1.289846
  1       1.289846
  2       1.289846
  3       1.289846
  4       1.289846
            ...   
  6420    1.166972
  6421    1.166972
  6422    1.166972
  6423    1.166972
  6424    1.166972
  Length: 6425, dtype: float64]]

In [13]:
async def collect_data_row(liquidation_evl_row, reserves_status, block_before=1000, mc_amount = 100, step = 100):
    
    until_block_num = liquidation_evl_row['block_num'] - block_before
    until_index = 9999
    target_address = liquidation_evl_row['on_behalf_of']
    until_block_n_index = combine_block_n_index({'block_num': until_block_num, 'index': until_index})
    liquidation_index = until_index

    liquidation_df = await get_liquidation_call(target_address)
    interaction_df = await get_interaction_data(target_address)

    interaction_df = interaction_df['action block_num index on_behalf_of reserve amount rate_mode rate'.split(' ')].copy()
    
    liquidation_df = liquidation_df[[
        'block_num', 'index', 'on_behalf_of', 
        'collateral_asset', 'debt_asset', 'debt_to_cover', 'liquidated_collateral_amount',
        'liquidator', 'receive_atoken']].copy()

    interaction_df['block_n_index'] = interaction_df.apply(combine_block_n_index, axis=1)
    
    liquidation_df['block_n_index'] = liquidation_df.apply(combine_block_n_index, axis=1)

    interaction_df = interaction_df.sort_values('block_n_index').reset_index(drop=True)
    

    # just give a random reserve address, will be swich in the following part
    for index in interaction_df.index:
        if interaction_df.loc[index, 'action'] == "LiquidationCall":
            interaction_df.loc[index, 'reserve'] = '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'

    # merge
    interaction_df['reserve'] = interaction_df['reserve'].apply(change_token_address_to_name).reset_index(drop=True)
    
    user_df = interaction_df.merge(reserves_status, on=['reserve'], how='left')
    
    user_df['reserve'] = user_df['reserve'].apply(change_token_address_to_name).reset_index(drop=True)
    

    liquidation_df['collateral_asset'] = liquidation_df['collateral_asset'].apply(change_token_address_to_name)
    liquidation_df['debt_asset'] = liquidation_df['debt_asset'].apply(change_token_address_to_name)
    
    if 'usdt' in interaction_df[interaction_df['action']=='Deposit']['reserve'].to_list():
        return

    def get_liquidation_data(df_row):
        df_row = df_row.copy()
        if df_row['action'] != 'LiquidationCall': return df_row
        # collateral
        block_n_index_x = df_row['block_n_index_x']
        liquidation_row = liquidation_df[liquidation_df['block_n_index'] == block_n_index_x]
        collateral_asset = liquidation_row['collateral_asset'].values[0]
        tmp_reserves_status = reserves_status[\
            (reserves_status['reserve'] == collateral_asset) &\
            (reserves_status['block_n_index'] <= block_n_index_x)].copy().sort_values('block_n_index')
        tmp_reserves_status = tmp_reserves_status.iloc[-1, :]

        df_row['block_num_y'] = tmp_reserves_status['block_num']
        df_row['index_y'] = tmp_reserves_status['index']
        df_row['liquidity_rate'] = tmp_reserves_status['liquidity_rate']
        df_row['liquidity_index'] = tmp_reserves_status['liquidity_index']
        df_row['block_n_index_y'] = tmp_reserves_status['block_n_index']

        debt_asset = liquidation_row['debt_asset'].values[0]
        tmp_reserves_status = reserves_status[\
            (reserves_status['reserve'] == debt_asset) &\
            (reserves_status['block_n_index'] <= block_n_index_x)].copy().sort_values('block_n_index')
        tmp_reserves_status = tmp_reserves_status.iloc[-1, :]

        df_row['stable_borrow_rate'] = tmp_reserves_status['stable_borrow_rate']
        df_row['variable_borrow_rate'] = tmp_reserves_status['variable_borrow_rate']
        df_row['variable_borrow_index'] = tmp_reserves_status['variable_borrow_index']
        
        return df_row

    from_df = user_df[user_df['block_n_index_y'] <= user_df['block_n_index_x']]
    from_df = from_df.loc[from_df.groupby('block_n_index_x').block_n_index_y.idxmax()].reset_index(drop=True)
    from_df = from_df.apply(get_liquidation_data, axis=1)

    collateral_dict = defaultdict(float)
    collatearl_able_dict = defaultdict(lambda :True)
    variable_debt_dict = defaultdict(float)
    stable_debt_dict = defaultdict(lambda : [None, None, None]) # amount, interest, start time

    liquidation_pool_target = [
        "ReserveUsedAsCollateralEnabled",
        "ReserveUsedAsCollateralDisabled", 
        "Deposit", 
        "Withdraw",
        "Borrow",
        "Repay",
        # "LiquidationCall",
        "Swap",
    ]

    SECONDS_PER_YEAR = 365 * 24 * 60 * 60
    RAY = 1e27

    ray_mul = lambda a,b: (a * b + RAY/2) / RAY
    ray_div = lambda a,b: (a * RAY + b/2) / b


    sub_interaction_df = interaction_df[interaction_df['block_n_index'] <= until_block_n_index].copy()

    __library = cdll.LoadLibrary('../eth_crawler/library.so')

    get_single_block_time = __library.get_single_block_time
    get_single_block_time.argtypes = [c_char_p, GoInt]
    get_single_block_time.restype = c_char_p

    # Block Time
    def get_block_time(block_num):
        try:
            res = get_single_block_time(
                archive_node.encode(), 
                GoInt(int(block_num))
            )
            res = res.decode("utf-8")
            res = json.loads(s=res)#.items()#, columns=['BlockNum', 'Timestamp'])
            
            return res[str(block_num)]
        except Exception as e: 
            print(e)


    def cal_stable_debt_change(stable_debt_amount_p, stable_borrow_rate_p, block_num, block_num_p):
        block_time = get_block_time(block_num)
        block_time_p = get_block_time(block_num_p)
        exp = block_time - block_time_p
        
        ###### Reference #####: https://etherscan.io/address/0xc6845a5c768bf8d7681249f8927877efda425baf#code
        expMinusOne = exp - 1
        expMinusTwo = exp - 2 if exp > 2 else 0
        ratePerSecond = stable_borrow_rate_p / SECONDS_PER_YEAR
        basePowerTwo = ray_mul(ratePerSecond, ratePerSecond) # (ratePerSecond * ratePerSecond + 0.5 * RAY)/RAY
        basePowerThree = ray_mul(basePowerTwo, ratePerSecond)#  + 0.5 * RAY)/RAY
        secondTerm = (exp * expMinusOne * basePowerTwo) / 2
        thirdTerm = (exp * expMinusOne * expMinusTwo * basePowerThree) / 6
        compounded_interest = RAY + (ratePerSecond * exp) + (secondTerm) + (thirdTerm)
        new_stable_balance = ray_mul(stable_debt_amount_p, compounded_interest)
        balance_increase = new_stable_balance - stable_debt_amount_p
        ########################################################################

        return new_stable_balance, balance_increase

    def update_target_debt_data(action_i, block_num, amount_i, token_name_i, 
            rate_mode_i, liquidity_index, variable_borrow_index, stable_borrow_rate):
        
        
        a_token_amount_i = ray_div(amount_i, liquidity_index)
        
        variable_debt_amount_i = ray_div(amount_i, variable_borrow_index)

        # For Stable Debt
        stable_debt_amount_i = amount_i #/ stable_borrow_rate
        stable_debt_amount_p, stable_borrow_rate_p, block_num_p = stable_debt_dict[token_name_i]
        if stable_debt_amount_p != None:
            new_stable_balance, balance_increase = cal_stable_debt_change(stable_debt_amount_p, stable_borrow_rate_p, block_num, block_num_p)

        
        if action_i == "ReserveUsedAsCollateralEnabled":
            collatearl_able_dict[token_name_i] = True
        elif action_i == "ReserveUsedAsCollateralDisabled":
            collatearl_able_dict[token_name_i] = False
        elif action_i == "Deposit":
            if collatearl_able_dict[token_name_i] == False and collateral_dict[token_name_i] == 0:
                collatearl_able_dict[token_name_i] = True
            collateral_dict[token_name_i] += a_token_amount_i
        elif action_i == 'Withdraw':
            if (collateral_dict[token_name_i] - a_token_amount_i) < 0:
                return False, np.abs(collateral_dict[token_name_i] - a_token_amount_i)
            collateral_dict[token_name_i] -= a_token_amount_i
        elif action_i == "Borrow":
            if rate_mode_i == '1': # stable
                if stable_debt_dict[token_name_i][0] is None:
                    stable_debt_dict[token_name_i] = [stable_debt_amount_i, stable_borrow_rate, block_num]
                else:
                    stable_debt_dict[token_name_i] = [new_stable_balance + stable_debt_amount_i, stable_borrow_rate, block_num]
            elif rate_mode_i == '2': # variable
                variable_debt_dict[token_name_i] += variable_debt_amount_i
            else:
                assert False, "rate_mode_i error"
        elif action_i == "Repay":
            if rate_mode_i == '1':
                if (new_stable_balance - stable_debt_amount_i) < 0:
                    return False, np.abs(new_stable_balance - stable_debt_amount_i)
                stable_debt_dict[token_name_i] = [new_stable_balance - stable_debt_amount_i, stable_borrow_rate, block_num]
            elif rate_mode_i == '2': # variable
                if (variable_debt_dict[token_name_i] - variable_debt_amount_i) < 0:
                    return False, np.abs(variable_debt_dict[token_name_i] - variable_debt_amount_i)
                variable_debt_dict[token_name_i] -= variable_debt_amount_i
        elif action_i == 'RebalanceStableBorrowRate':
            stable_debt_dict[token_name_i] = [new_stable_balance, stable_borrow_rate, block_num]
        else:
            assert False, "Interaction Data error"

        return True, 0

    def get_token_value(block_num, index, fix_decimal=False):
        collateral_in_original_unit = defaultdict(float)
        var_debt_in_original_unit = defaultdict(float)
        sta_debt_in_original_unit = defaultdict(float)

        block_n_index = combine_block_n_index(dict(block_num=block_num, index=index))

        for token_name, able in collatearl_able_dict.items():
            decimal_fixer = 1
            if able:
                tmp_status = reserves_status[(reserves_status['reserve'] == token_name) &\
                    (reserves_status['block_n_index'] <= block_n_index)].copy().sort_values('block_n_index').iloc[-1,:]
                if fix_decimal:
                    decimal_fixer = 10 ** decimal_dict[token_name]
                collateral_in_original_unit[token_name] = ray_mul(collateral_dict[token_name], tmp_status["liquidity_index"]) / decimal_fixer
        
        for token_name, able in variable_debt_dict.items():
            decimal_fixer = 1
            tmp_status = reserves_status[(reserves_status['reserve'] == token_name) &\
                    (reserves_status['block_n_index'] <= block_n_index)].copy().sort_values('block_n_index').iloc[-1,:]
            if fix_decimal:
                decimal_fixer = 10 ** decimal_dict[token_name]
            var_debt_in_original_unit[token_name] = ray_mul(variable_debt_dict[token_name], tmp_status["variable_borrow_index"]) / decimal_fixer

        for token_name, stable_debt in stable_debt_dict.items():
            decimal_fixer = 1
            if stable_debt[0] is not None:
                stable_debt_amount_p, stable_borrow_rate_p, block_num_p = stable_debt_dict[token_name]
                new_stable_balance, balance_increase = cal_stable_debt_change(stable_debt_amount_p, stable_borrow_rate_p, block_num, block_num_p)
                if fix_decimal:
                    decimal_fixer = 10 ** decimal_dict[token_name]
                sta_debt_in_original_unit[token_name] = new_stable_balance / decimal_fixer
        
        
        
        return collateral_in_original_unit, var_debt_in_original_unit, sta_debt_in_original_unit


    for index_i in sub_interaction_df.index:

        action_i = sub_interaction_df.loc[index_i, 'action']
        block_n_index = sub_interaction_df.loc[index_i, 'block_n_index']
        block_num = sub_interaction_df.loc[index_i, 'block_num']
        index = sub_interaction_df.loc[index_i, 'index']

        # block_time = get_block_time(block_num)
        before_data = from_df[from_df['block_n_index_x'] == block_n_index]#['amount'].values[0]
        liquidity_index = before_data['liquidity_index'].values[0]
        variable_borrow_index = before_data['variable_borrow_index'].values[0]
        stable_borrow_rate = before_data['stable_borrow_rate'].values[0]
        
        if action_i == "LiquidationCall":
            'collateral_asset', 'debt_asset', 'debt_to_cover', 'liquidated_collateral_amount',
            liquidation_i = liquidation_df[liquidation_df['block_n_index'] == block_n_index].copy().reset_index(drop=True)
            collateral_asset = liquidation_i.loc[0, 'collateral_asset']
            debt_asset = liquidation_i.loc[0, 'debt_asset']
            debt_to_cover = liquidation_i.loc[0, 'debt_to_cover']
            liquidated_collateral_amount = liquidation_i.loc[0, 'liquidated_collateral_amount']

            # a_token_amount_i = ray_div(liquidated_collateral_amount, liquidity_index)
            # collateral_dict[collateral_asset] -= a_token_amount_i

            collateral_in_original_unit, var_debt_in_original_unit, sta_debt_in_original_unit = get_token_value(block_num, index)

            if var_debt_in_original_unit[debt_asset] < debt_to_cover:
                var_debt_to_liquidate = var_debt_in_original_unit[debt_asset]
                sta_debt_to_repay = debt_to_cover - var_debt_to_liquidate
            else:
                var_debt_to_liquidate = debt_to_cover
                sta_debt_to_repay = 0

            success, remaining_token = update_target_debt_data(
                "Repay", block_num, var_debt_to_liquidate, debt_asset, 
                "2", liquidity_index, variable_borrow_index, stable_borrow_rate)
            assert success

            if sta_debt_to_repay > 0:
                success, remaining_token = update_target_debt_data(
                    "Repay", block_num, sta_debt_to_repay, debt_asset, 
                    "1", liquidity_index, variable_borrow_index, stable_borrow_rate)
                assert success
            
            success, remaining_token = update_target_debt_data(
                    "Withdraw", block_num, liquidated_collateral_amount, collateral_asset, 
                    "-1", liquidity_index, variable_borrow_index, stable_borrow_rate)
            assert success

        else:
            amount_i = sub_interaction_df.loc[index_i, 'amount']
            token_name_i = sub_interaction_df.loc[index_i, 'reserve']
            rate_mode_i = sub_interaction_df.loc[index_i, 'rate_mode']

            update_target_debt_data(action_i, block_num, amount_i, token_name_i, 
            rate_mode_i, liquidity_index, variable_borrow_index, stable_borrow_rate)
    token_value_dicts = get_token_value(until_block_num, liquidation_index, fix_decimal=True)
    token_value_dicts = {i:j for i,j in zip(['collateral', 'var_debt', 'sta_debt'], token_value_dicts)}


    price_data = await get_price_data(until_block_num, previous_block=6424)
    price_data['token0'] = price_data['token0'].apply(lambda x: 'weth' if x == 'eth' else x)
    price_data['token1'] = price_data['token1'].apply(lambda x: 'weth' if x == 'eth' else x)
    price_data = price_data[['block_num', 'oracle_name', 'token0', 'token1', 'current']]

    block_num_df = pd.DataFrame(
        range(
            price_data.block_num.min(), 
            until_block_num + 1
        ),
        columns=['block_num']
    )
    block_num_df.set_index('block_num', inplace=True)

    uniswapv3_price_dict = {}
    for token in ['usdt', 'dai', 'usdc']:
        sub_price_df = price_data[(price_data['oracle_name'] == 'uniswapv3') & (price_data['token1'] == token)].copy()
        sub_price_df[f'{token}'] = 1/sub_price_df['current']
        sub_price_df.set_index('block_num', inplace=True)
        sub_price_df = sub_price_df.merge(block_num_df, how='right', left_index=True, right_index=True)
        sub_price_df.fillna(method='ffill', inplace=True)
        sub_price_df.fillna(method='bfill', inplace=True)
        uniswapv3_price_dict[token] = sub_price_df[token]
    chainlink_price_dict ={}
    for token in ['usdt', 'dai', 'usdc']:
        sub_price_df = price_data[(price_data['oracle_name'] == 'chainlink') & (price_data['token0'] == token)].copy()
        sub_price_df[f'{token}'] = sub_price_df['current']
        sub_price_df.set_index('block_num', inplace=True)
        sub_price_df = sub_price_df.merge(block_num_df, how='right', left_index=True, right_index=True)
        sub_price_df.fillna(method='ffill', inplace=True)
        sub_price_df.fillna(method='bfill', inplace=True)
        chainlink_price_dict[token] = sub_price_df[token]


    collatearl_in_eth = 0
    debt_in_eth = 0
    for token_name, token_amount in token_value_dicts['collateral'].items():
        if token_name == 'weth':
            collatearl_in_eth += token_amount
        else:
            collatearl_in_eth += token_amount * chainlink_price_dict[token_name].loc[until_block_num]

    for token_name, token_amount in token_value_dicts['var_debt'].items():
        if token_name == 'weth':
            debt_in_eth += token_amount
        else:
            debt_in_eth += token_amount * chainlink_price_dict[token_name].loc[until_block_num]

    for token_name, token_amount in token_value_dicts['sta_debt'].items():
        if token_name == 'weth':
            debt_in_eth += token_amount
        else:
            debt_in_eth += token_amount * chainlink_price_dict[token_name].loc[until_block_num]

    collatearl_m_threshold_in_eth = 0
    debt_m_threshold_in_eth = 0
    for token_name, token_amount in token_value_dicts['collateral'].items():
        if token_name == 'weth':
            collatearl_m_threshold_in_eth += token_amount * liquidation_threshold_dict[token_name]
        else:
            if liquidation_threshold_dict[token_name] is None:
                print(123)
            collatearl_m_threshold_in_eth += token_amount * chainlink_price_dict[token_name].loc[until_block_num]  * liquidation_threshold_dict[token_name]

    for token_name, token_amount in token_value_dicts['var_debt'].items():
        if token_name == 'weth':
            debt_m_threshold_in_eth += token_amount
        else:
            debt_m_threshold_in_eth += token_amount * chainlink_price_dict[token_name].loc[until_block_num]

    for token_name, token_amount in token_value_dicts['sta_debt'].items():
        if token_name == 'weth':
            debt_m_threshold_in_eth += token_amount
        else:
            debt_m_threshold_in_eth += token_amount * chainlink_price_dict[token_name].loc[until_block_num]
    collatearl_m_threshold_in_eth, debt_m_threshold_in_eth

    HF = (collatearl_m_threshold_in_eth/debt_m_threshold_in_eth)  


    used_token_list = []
    price_data_list = []
    price_name_list = []
    for asset_from in ['collateral', 'var_debt', 'sta_debt']:
        for token_name in ['usdc', 'usdt', 'dai']:
        # for token_name, token_amount in token_value_dicts[asset_from].items():
            if token_name == 'weth': continue
            if token_name not in used_token_list:
                used_token_list.append(token_name)
                price_data_list.append(chainlink_price_dict[token_name])
                price_name_list.append(f'chainlink_{token_name}')
                price_data_list.append(uniswapv3_price_dict[token_name])
                price_name_list.append(f'uniswapv3_{token_name}')
    var_train_df = pd.concat(price_data_list, axis=1)
    var_train_df.columns = price_name_list
    var_train_df = var_train_df.reset_index(drop=False)

    train_test_split = 1
    train_data = var_train_df[:(int(len(var_train_df)*train_test_split)+1)].set_index('block_num')
    test_data = var_train_df[(int(len(var_train_df)*train_test_split)+1):].set_index('block_num')
    tmp_df = train_data.copy()#.set_index('block_num')
    # log_df = np.log(tmp_df)
    log_f_diff = tmp_df.diff().dropna() 
    log_f_diff = log_f_diff.reset_index(drop=True)
    trained_var = get_var_result(log_f_diff)              


    def mc_simulate(df, step=240):
        price_diff_prediction = pd.DataFrame(trained_var.simulate_var(step), columns=log_f_diff.columns)
        price_prediction = invert_transformation(tmp_df, price_diff_prediction) 
        hf_df = cal_hf(price_prediction, token_value_dicts, liquidation_threshold_dict)
        return hf_df
    
    
    mc_hf = pd.DataFrame(range(mc_amount)).apply(mc_simulate, args=(step,), axis=1).T
    potential_liquidation = mc_hf.apply(lambda x: (x < 1).any(), axis=0)
    mc_hf_pct = mc_hf.apply(cal_pct_be_liquidated, axis=1)

    # mc_hf = pd.DataFrame(range(mc_amount)).parallel_apply(mc_simulate, args=(step,), axis=1).T
    # potential_liquidation = mc_hf.parallel_apply(lambda x: (x < 1).any(), axis=0)
    # mc_hf_pct = mc_hf.parallel_apply(cal_pct_be_liquidated, axis=1)
    # print(tmp_df, token_value_dicts, liquidation_threshold_dict)
    actual_hf = cal_hf(tmp_df.reset_index(), token_value_dicts, liquidation_threshold_dict)

    return [liquidation_evl_row, mc_hf_pct, actual_hf]

In [16]:
reserves_status = await get_reserves_status()
reserves_status = reserves_status[[
        'reserve', 'block_num', 'index',  
        'liquidity_rate', 'stable_borrow_rate', 'variable_borrow_rate', 
        'liquidity_index','variable_borrow_index'
]].copy()
reserves_status['block_n_index'] = reserves_status.apply(combine_block_n_index, axis=1)
reserves_status = reserves_status.sort_values('block_n_index').reset_index(drop=True)
reserves_status['reserve'] = reserves_status['reserve'].apply(change_token_address_to_name).reset_index(drop=True)

In [21]:
ttt = liquidation_evl_df.iloc[:10].apply(collect_data_row, args=(reserves_status,), axis=1)

In [30]:
ttt[0].send(None)

<Future pending cb=[_chain_future.<locals>._call_check_cancel() at /Users/anl/opt/anaconda3/envs/test2/lib/python3.8/asyncio/futures.py:360]>

In [ ]:
async def collect_data_row(liquidation_evl_row, reserves_status, block_before=1000, mc_amount = 100, step = 100):

In [ ]:
import time
from multiprocessing.pool import Pool

def numsCheng(i):
    return i * 2


time1 = time.time()
nums_list = [1, 2, 3, 4, 5, 6, 7, 8, 9]
pool = Pool(processes=5)
result = pool.map(numsCheng, nums_list)
pool.close()        # 关闭进程池，不再接受新的进程
pool.join()         # 主进程阻塞等待子进程的退出
